### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Underweight/GSE163901'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine the availability of gene expression data
is_gene_available = True  # Assuming the dataset contains gene expression data

# Determine the availability and the corresponding key for 'Underweight' trait
for key, values in sample_characteristics.items():
    if any('bmi code:' in value for value in values):
        bmi_key = key
        unique_bmi_codes = set(value.split(': ')[1].strip() for value in values)
        if len(unique_bmi_codes) > 1:
            trait_row = bmi_key
        break

# Determine the availability and the corresponding key for 'age'
for key, values in sample_characteristics.items():
    if any("mother's age:" in value for value in values):
        age_key = key
        unique_ages = set(value.split(': ')[1].strip() for value in values)
        if len(unique_ages) > 1:
            age_row = age_key
        break

# Determine the availability and the corresponding key for 'gender'
for key, values in sample_characteristics.items():
    if any("infant's sex:" in value for value in values):
        gender_key = key
        unique_genders = set(value.split(': ')[1].strip() for value in values)
        if len(unique_genders) > 1:
            gender_row = gender_key
        break

# Define the conversion functions for each variable

def convert_trait(value):
    try:
        val = value.split(': ')[1].strip()
        if val == 'UNDERWEIGHT':
            return 1
        elif val in ['NORMAL', 'OVERWEIGHT', 'OBESE']:
            return 0
    except:
        pass
    return None

def convert_age(value):
    try:
        val = value.split(': ')[1].strip()
        if val == 'NA':
            return None
        return float(val)
    except:
        pass
    return None

def convert_gender(value):
    try:
        val = value.split(': ')[1].strip()
        if val == '0':
            return 0
        elif val == '1':
            return 1
    except:
        pass
    return None

# Save cohort information
save_cohort_info('GSE163901', './preprocessed/Underweight/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is available, carry out clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Underweight', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Underweight/trait_data/GSE163901.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
